In [1]:
f_func <- "./functions.ipynb"
eval(parse(text = system2('jupyter', c('nbconvert', f_func, '--to=script', '--stdout'), stdout = TRUE)))

Loading required package: Rsamtools

Loading required package: GenomeInfoDb

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, aperm, append, as.data.frame, basename, cbind,
    colnames, dirname, do.call, duplicated, eval, evalq, Filter, Find,
    get, grep, grepl, intersect, is.unsorted, lapply, Map, mapply,
    match, mget, order, paste, pmax, pmax.int, pmin, pmin.int,
    Position, rank, rbind, Reduce, rownames, sapply, setdiff, sort,
    table, tapply, union, unique, unsplit, which.max, which.min


Loading required package: S4Vectors

Loading required package: stats4


Attaching package: ‘S4Vectors’


The following objects are masked from ‘package:base’:

    expand.grid, I, unname


Loading required package: IRanges

Loading required package: GenomicRanges

Loading required package: Biostrings



The following object is masked from ‘package:base’:

    as.matrix


Warning message:
“no DISPLAY variable so Tk is not available”
Registered S3 method overwritten by 'quantmod':
  method            from
  as.zoo.data.frame zoo 

Loaded TSdist v3.7.1. See ?TSdist for help, citation("TSdist") for use in publication.


Loading required package: dtplyr

Loading required package: profvis

Loading required package: ggforce

Loading required package: pracma


Attaching package: ‘pracma’


The following object is masked from ‘package:purrr’:

    cross


The following object is masked from ‘package:Hmisc’:

    ceil


Loading required package: rstatix


Attaching package: ‘rstatix’


The following object is masked from ‘package:IRanges’:

    desc


The following object is masked from ‘package:stats’:

    filter


Loading required package: ggpubr

Loading required package: jsonlite


Attaching package: ‘jsonlite’


The following object is masked from ‘package:R.utils’:

    validate


The fo

In [8]:
DIR_IN_HUMAN="../../output/cfspi/samples/results/host_TLEN_readCount/"
samples_metadata <- read.csv(file = "../../output/samples_metadata.csv") %>% as.data.table()

list.files(DIR_IN_HUMAN, pattern = "_MT_TLEN_EndMotif.txt") %>% length()

list.files(DIR_IN_HUMAN, pattern = "[pos/neg]_MT_TLEN_EndMotif.txt") 

[1] 516

[1] "A01Basp_R1neg_MT_TLEN_EndMotif.txt"  
 [2] "A01Basp_R1pos_MT_TLEN_EndMotif.txt"  
 [3] "A01Basp_R2neg_MT_TLEN_EndMotif.txt"  
 [4] "A01Basp_R2pos_MT_TLEN_EndMotif.txt"  
 [5] "A01BaspK_R1neg_MT_TLEN_EndMotif.txt" 
 [6] "A01BaspK_R1pos_MT_TLEN_EndMotif.txt" 
 [7] "A01BaspK_R2neg_MT_TLEN_EndMotif.txt" 
 [8] "A01BaspK_R2pos_MT_TLEN_EndMotif.txt" 
 [9] "A01Pasp_R1neg_MT_TLEN_EndMotif.txt"  
[10] "A01Pasp_R1pos_MT_TLEN_EndMotif.txt"  
[11] "A01Pasp_R2neg_MT_TLEN_EndMotif.txt"  
[12] "A01Pasp_R2pos_MT_TLEN_EndMotif.txt"  
[13] "A01PaspK_R1neg_MT_TLEN_EndMotif.txt" 
[14] "A01PaspK_R1pos_MT_TLEN_EndMotif.txt" 
[15] "A01PaspK_R2neg_MT_TLEN_EndMotif.txt" 
[16] "A01PaspK_R2pos_MT_TLEN_EndMotif.txt" 
[17] "A01Pctrl_R1neg_MT_TLEN_EndMotif.txt" 
[18] "A01Pctrl_R1pos_MT_TLEN_EndMotif.txt" 
[19] "A01Pctrl_R2neg_MT_TLEN_EndMotif.txt" 
[20] "A01Pctrl_R2pos_MT_TLEN_EndMotif.txt" 
[21] "A01PctrlK_R1neg_MT_TLEN_EndMotif.txt"
[22] "A01PctrlK_R1pos_MT_TLEN_EndMotif.txt"
[23] "A01PctrlK_R2neg_MT_TLEN_EndMotif.txt"
[24] "A01PctrlK_R2pos_MT_TLEN_EndMotif.txt"
[25] "A02Basp_R1neg_MT_TLEN_EndMotif.txt"  
[26] "A02Basp_R1pos_MT_TLEN_EndMotif.txt"  
[27] "A02Basp_R2neg_MT_TLEN_EndMotif.txt"  
[28] "A02Basp_R2pos_MT_TLEN_EndMotif.txt"  
[29] "A02BaspK_R1neg_MT_TLEN_EndMotif.txt" 
[30] "A02BaspK_R1pos_MT_TLEN_EndMotif.txt"

In [9]:
if (exists("host_MT_IS_meta")){rm("host_MT_IS_meta")}
profvis({
for (file in list.files(DIR_IN_HUMAN, pattern = "[pos/neg]_MT_TLEN_EndMotif.txt")){
    sample_tmp <- unlist(strsplit(file, "_"))[1] 
    side_tmp <- unlist(strsplit(file, "_"))[2]
    
    if(countLines(paste0(DIR_IN_HUMAN, file)) > 1){
        pre_tmp_IS <- fread(paste0(DIR_IN_HUMAN, file), sep = ",") %>% 
                    dplyr::rename(EndMotif_tmp = EndMotif) %>% 
                    mutate(sample_id = sample_tmp) %>% 
                    mutate(side = side_tmp) %>% 
                    mutate(read = substr(side_tmp, 0, 2)) %>% 
                    filter(TLEN != 0) %>% #remove TLEN not equal to 0
                    filter(abs(TLEN) < 1000) %>% #filter max TLEN of 1000
                    filter(!grepl(EndMotif_tmp, pattern = "N")) %>% #remove reads with end-motif including "N"
                    group_by(TLEN, EndMotif_tmp, sample_id, side) %>% 
                    summarise(Count = sum(Count), .groups = "keep")
        
        if (grepl(side_tmp, pattern = "neg")){
            tmp_IS <- pre_tmp_IS #%>% select(-any_of("EndMotif_tmp"))
            tmp_IS$EndMotif <- as.character(reverseComplement(DNAStringSet(pre_tmp_IS$EndMotif_tmp))) 
        } else {
            tmp_IS <- pre_tmp_IS #%>% select(-any_of("EndMotif_tmp"))
            tmp_IS$EndMotif <- pre_tmp_IS$EndMotif_tmp
        }
        
        tmp_IS_meta <- merge(x = samples_metadata, y = tmp_IS, by.x = "sample_id", by.y = "sample_id") 
        if (!exists("host_MT_IS_meta")){host_MT_IS_meta <- tmp_IS_meta} else {
            host_MT_IS_meta <- rbind(host_MT_IS_meta, tmp_IS_meta)}
    }
    fwrite(host_MT_IS_meta, "../../output/PP_figures/host_new_MT_IS_meta.csv", row.names = FALSE)
}
})

HTML widgets cannot be represented in plain text (need html)